In [1]:
import pystac_client
import pystac
import odc.stac
import geopandas
import xarray 
import rioxarray
import planetary_computer
import pathlib
import pandas
import numpy
import folium
import dask.distributed
import branca.element, branca.colormap # Remove whitespace around small folium map

module_path = pathlib.Path.cwd() / '..' / 'scripts'
import sys
if str(module_path) not in sys.path:
    sys.path.append(str(module_path))
import utils

%load_ext autoreload
%autoreload 2


# Ensure geometries setup

In [23]:
tiles_south_island = utils.create_tiles_south_island(distance_offshore = 3_000, tile_length = 10_000)
tiles_chatham_island = utils.create_tiles_chatham_island(distance_offshore = 3_000, tile_length = 10_000)
tiles_north_island = utils.create_tiles_north_island(distance_offshore = 3_000, tile_length = 10_000)
test_sites = utils.create_test_sites(distance_offshore = 3_000)
land = geopandas.read_file(utils.DATA_PATH / "vectors" / f"main_islands.gpkg")

In [24]:
test_sites

,name,geometry
0,rakiora,"MULTIPOLYGON (((1231000 4787000, 1231000 47904..."
1,waikouaiti,"POLYGON ((1418619.3 4945964.99, 1418813.713 49..."
5,chatham,"POLYGON ((2449479.612 5076000, 2449477.966 507..."
2,akaroa,"MULTIPOLYGON (((1592296.284 5140000, 1592304.5..."
3,matau,"POLYGON ((1604281.608 5230000, 1604181.583 523..."
4,wellington,"POLYGON ((1753000 5419000, 1753000 5422664.171..."


In [25]:
test_sites.explore()

# Set up - call only once each session

In [3]:
client = dask.distributed.Client()
odc.stac.configure_rio(cloud_defaults=True, client=client)
display(client)

odc.stac.configure_rio(cloud_defaults=True, aws={"aws_unsigned": True})
catalogue = {"url": "https://planetarycomputer.microsoft.com/api/stac/v1",
             "collections": {"sentinel": "sentinel-2-l2a", "dem": "cop-dem-glo-30"}}
client = pystac_client.Client.open(catalogue["url"], modifier=planetary_computer.sign_inplace) 

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 31.73 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:61391,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 31.73 GiB
Comm: tcp://127.0.0.1:61412,Total threads: 2
Dashboard: http://127.0.0.1:61414/status,Memory: 7.93 GiB
Nanny: tcp://127.0.0.1:61394,


# Site of interest - change date, site name and thresholds

In [4]:
date_YYMM = "2016-09-19" # "2017-02"
site_name = "rakiora"
thresholds = {"min_ndvi": 0.03, "max_ndvi": 0.7, "max_ndwi": 0.1, "min_ndvri": 0.03, "max_ndwi2": -0.2,}

# Generate results

In [6]:
site_index = (test_sites["name"] == site_name).idxmax()
site_bbox = test_sites.to_crs(utils.CRS_WSG).loc[site_index].geometry.bounds

output_path = utils.DATA_PATH / "rasters" / "test_sites" / site_name
output_path.mkdir(parents=True, exist_ok=True)

data = utils.kelp_single_day(client, date_YYMM, roi=test_sites.loc[[site_index]], thresholds)
data.to_netcdf(output_path / f'{date_YYMM}.nc', format="NETCDF4", engine="netcdf4")

		Ocean cloud percentage ['0.62%']


Load back in to avoid memory errors and create polygons

In [7]:
data = rioxarray.rioxarray.open_rasterio(output_path / f'{date_YYMM}.nc', masked=True)#.squeeze("band", drop=True)
buffer = 3
kelp_polygons = utils.polygon_from_raster(data["kelp"])
kelp_polygons_buffered = kelp_polygons.buffer(-utils.RASTER_DEFAULTS["resolution"] * buffer).buffer(utils.RASTER_DEFAULTS["resolution"] * buffer)
ocean_buffered = test_sites.loc[[site_index]].to_crs(kelp_polygons.crs)
ocean_buffered = ocean_buffered.overlay(kelp_polygons, how="difference")
ocean_buffered = ocean_buffered.buffer(-utils.RASTER_DEFAULTS["resolution"] * buffer)

Saves out histograms in the same folder as the data of the ocean and kelp regions

In [13]:
utils.plot_hists_single_day(data, date_YYMM, ocean_buffered, output_path)

In [14]:
# Put it all on a single interactive map
center = [(site_bbox[1] + site_bbox[3]) / 2, (site_bbox[0] + site_bbox[2]) / 2]  # Assuming min_lon and max_lon are defined
m = folium.Map(location=center, zoom_start=13)

# RGB
data.odc.to_rgba(bands=utils.get_band_names_from_common(["red", "green", "blue"]), vmin=0, vmax=1000).odc.add_to(m, name="Satellite RBG")


# Categorical for the predicted classes and for the training data
data["kelp"].odc.add_to(m, name="Kelp")

kelp_polygons.explore(m=m, color="magenta", style_kwds={"fillOpacity": 0}, name="kelp")
kelp_polygons_buffered.explore(m=m, color="red", style_kwds={"fillOpacity": 0}, name="kelp buffered dialated")
ocean_buffered.explore(m=m, color="blue", style_kwds={"fillOpacity": 0}, name="ocean dialated")

# Layer control
folium.LayerControl().add_to(m)
import folium.plugins
folium.plugins.MousePosition(position="topright",).add_to(m)

m